### Data Retrieval from Fan Market Cap API

This script aims to fetch and process fan token data from the Fan Market Cap API.

#### Steps Involved:

1. **Library Import**:
    - Essential Python libraries, `requests` and `pandas`, are imported to handle API requests and data manipulation.

2. **Initial Setup**:
    - The base URL for the API is defined.
    - A specific `days_value` is set to "max", indicating the maximum available historical data will be fetched.
    - API headers, including the API key, are specified for authentication.

3. **Specifying Data Interests**:
    - A list of `slugs` representing different fan tokens (like FC Barcelona, Manchester City, etc.) is prepared. This list indicates the specific fan tokens of interest.

4. **Data Collection**:
    - For each slug in the list, an API request is made to fetch the associated data.
    - If the API response is successful, the data is:
        - Converted to a DataFrame.
        - Enhanced with additional columns like `slug`, `date`,


In [11]:
#importing packages and libraries 
import requests
import pandas as pd

# Initial setup from the provided code
base_url = "https://api.fanmarketcap.com/api/v1/markets/"
days_value = "max"  # Available values are 7d, 30d, 60d, 90d

headers = {
    'X-API-KEY': '' #insert API key 
}

# List of slugs to fetch data for
slugs = [
    "fc-barcelona-fan-token", "galatasaray-fan-token", "as-roma-fan-token", 
    "manchester-city-fan-token", "lazio-fan-token", "paris-saint-germain-fan-token", 
    "fc-porto-fan-token", "juventus-fan-token", "ac-milan-fan-token", "arsenal-fan-token", 
    "atletico-madrid-fan-token", "trabzonspor-fan-token", "inter-milan-fan-token", 
    "napoli-fan-token", "valencia-fan-token", "young-boys-fan-token", "legia-warsaw-fan-token", 
    "real-sociedad-fan-token", "as-monaco-fan-token", "sevilla-fc-fan-token", "fenerbahce-fan-token", 
    "besiktas-fan-token", "aston-villa-fan-token", "everton-fc-fan-token", "dinamo-zagreb-fan-token", 
    "leeds-united-fan-token", "bologna-fc-fan-token", "udinese-fan-token"
]

# Dictionary to store data for each slug
data_dict = {}

# List to store individual dataframes
df_list = []

# Loop through each slug and make the API call
for slug in slugs:
    endpoint_url = f"{base_url}{slug}/{days_value}/"
    response = requests.get(endpoint_url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        # Convert the data to a dataframe
        df = pd.DataFrame(data)
    
        df_list.append(df)

        # Add a new column 'slug' to the dataframe
        df['slug'] = slug

        # Convert the date format
        df['date'] = pd.to_datetime(df['date'], unit='ms').dt.strftime('%Y-%m-%d')
        
        # Create the 'token_name' column
        df['token_name'] = df['slug'].str.replace('-', ' ')

          # Create the 'team_name' column
        df['team_name'] = df['token_name'].str.replace('fan token', '').str.strip()
        
    else:
        print(f"Error for slug {slug}: {response.status_code}: {response.text}")

# Concatenate all the dataframes in the list into a single dataframe
fan_tokens_df = pd.concat(df_list, ignore_index=True)

# Remove the 'market_cap' column
fan_tokens_df = fan_tokens_df.drop(columns=['market_cap'])

# Reordering columns 
fan_tokens_df = fan_tokens_df[['team_name', 'token_name', 'slug', 'date', 'volume', 'price']]


In [12]:
fan_tokens_df

,team_name,token_name,slug,date,volume,price
0,fc barcelona,fc barcelona fan token,fc-barcelona-fan-token,2020-06-25,311876.954013,6.261206
1,fc barcelona,fc barcelona fan token,fc-barcelona-fan-token,2020-06-26,311876.954013,6.261206
2,fc barcelona,fc barcelona fan token,fc-barcelona-fan-token,2020-06-27,300380.939093,5.804014
3,fc barcelona,fc barcelona fan token,fc-barcelona-fan-token,2020-06-28,318666.015727,4.971234
4,fc barcelona,fc barcelona fan token,fc-barcelona-fan-token,2020-06-29,331790.297430,5.050388
...,...,...,...,...,...,...
21796,udinese,udinese fan token,udinese-fan-token,2023-10-01,60013.874586,0.966765
21797,udinese,udinese fan token,udinese-fan-token,2023-10-02,56180.396510,0.986123
21798,udinese,udinese fan token,udinese-fan-token,2023-10-03,52872.125087,0.952171
21799,udinese,udinese fan token,udinese-fan-token,2023-10-04,49497.937829,0.898068


### Snowflake Data Insertion: Batched Approach

This code is designed to efficiently insert data from a pandas DataFrame, `fan_tokens_df`, into a Snowflake database table named `fan_tokens`.

1. **Cursor Initialization:** 
    - A cursor object, `cur`, is created using the established connection, `conn`.
    
2. **Query Definition:** 
    - An SQL `INSERT` statement is prepared to insert records into the `fan_tokens` table.
    
3. **Parameter Preparation:** 
    - The DataFrame `fan_tokens_df` is converted into a list of dictionaries, where each dictionary represents a record to be inserted.
    
4. **Batch Insertion:** 
    - Due to Snowflake's limitation on the number of expressions in a list (16,384), the insertion process is batched. 
    - The `CHUNK_SIZE` is set to 16,000 (a safe value below the limit) and the code inserts data in chunks to ensure we stay within Snowflake's constraints.
    
5. **Fetching Results:** 
    - After the insertion, any results (if available) from the last operation are fetched and printed.
    
6. **Cleanup:** 
    - The cursor and the connection to the database are closed for resource optimization.


In [3]:
# Importing connector
import snowflake.connector

#setting credentials
import os
os.environ["SNOWSQL_USR"] = '' #insert username
os.environ["SNOWSQL_PWD"] = '' #insert password
os.environ["SNOWSQL_ACC"] = 'sx14805.west-europe.azure'
os.environ["SNOWSQL_WH"] = 'COMPUTE_WH'
os.environ["SNOWSQL_DB"] = 'PYTHON_PROJECT'
os.environ["SNOWSQL_SCH"] = 'PUBLIC'

### Connecting to Snowflake
`Warning: If MFA is enabled, Snowflake will request authentication with the selected method (push notification, text message or phone call)`

In [2]:
# Connect to Snowflake
conn = snowflake.connector.connect(
    user=os.getenv('SNOWSQL_USR'),
    password=os.getenv('SNOWSQL_PWD'),
    account=os.getenv('SNOWSQL_ACC'),
    warehouse=os.getenv('SNOWSQL_WH'),
    database=os.getenv('SNOWSQL_DB'),
    schema=os.getenv('SNOWSQL_SCH')
)


NameError: name 'snowflake' is not defined

In [30]:
# Create a cursor object
cur = conn.cursor()

#Define the query
query = '''
INSERT INTO fan_tokens (team_name, token_name, slug, date, volume, price)
VALUES (%(team_name)s, %(token_name)s, %(slug)s, %(date)s, %(volume)s, %(price)s)
'''
params = fan_tokens_df.to_dict('records')

# Assuming CHUNK_SIZE is the size you want each batch to be
CHUNK_SIZE = 16000  # or any number less than 16,384

for i in range(0, len(params), CHUNK_SIZE):
    chunk = params[i:i + CHUNK_SIZE]
    cur.executemany(query, chunk)

# Fetch the results
results = cur.fetchall()

# Print the results
for row in results:
        print(row)

# Close the cursor and connection
cur.close()
conn.close()

(5801,)
